<a href="https://colab.research.google.com/github/joaograndotto/PIBITI/blob/main/Fiocruz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd

In [2]:
url = "https://raw.githubusercontent.com/joaograndotto/PIBITI/main/Datasets/dados_multi_classe.csv"
df = pd.read_csv(url, delimiter=',')

In [3]:
df

,Institution,Affiliations
0,aristotle university greece,1st department of ophthalmology aristotle univ...
1,aristotle university greece,aristotle univ thessaloniki greece
2,aristotle university greece,aristotle univ thessaloniki thessaloniki greece
3,aristotle university greece,aristotle university of thessaloniki greece
4,aristotle university greece,aristotle university of thessaloniki thessalon...
...,...,...
122929,universidade federal reconcavo,"univ fed reconcavo bahia, centro formacao prof..."
122930,unesp,unesp-marilia
122931,universidade federal campina grande,univ federal campina grande
122932,revista saude publica,revista de saude publica


In [4]:
fiocruz = df[df['Institution'].str.contains("fiocruz")]

In [5]:
fiocruz.shape[0]

20745

In [6]:
fiocruz.sample(n=10)

,Institution,Affiliations
24371,fiocruz mg rené rachou cpqrr,programa de pósgraduação em ciências da saúde ...
44409,ini instituto nacional de infectologia evandr...,laboratory of clinical epidemiology evandro ch...
14710,ensp escola nacional de saúde pública sérgio ...,fundação oswaldo cruz escola nacional de saúde...
14100,ensp escola nacional de saúde pública sérgio ...,escola nacional de saúde pública sérgio arouca...
43959,ini instituto nacional de infectologia evandr...,instituto de pesquisa clínica evandro chagas f...
42245,incqs instituto nacional de controle de quali...,postgraduation program in sanitary surveillanc...
22960,fiocruz mg rené rachou cpqrr,grupo integrado de pesquisa em biomarcadores c...
26155,fiocruz pe instituto aggeu magalhães cpqam,fundação oswaldo cruz fiocruz ufpe brazil
39443,iff instituto fernades figueira fiocruz,instituto fernandes figueira fiocruz departame...
18973,fiocruz am instituto leônidas e maria deane ...,instituto leônidas e maria deane fiocruz amaz...


In [7]:
df = fiocruz.to_csv("dicionario_fiocruz.tsv", sep = "\t", index=False)

In [9]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz, plot_tree
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
from sklearn.metrics import classification_report, confusion_matrix
from IPython.display import Image 
from six import StringIO
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [10]:
url = "https://raw.githubusercontent.com/joaograndotto/PIBITI/main/Datasets/dicionario_fiocruz.tsv"
df = pd.read_csv(url, delimiter='\t')

In [11]:
df.sample(n=3)

,Institution,Affiliations
7156,fiocruz ba centro de pesquisa gonçalo moniz ...,gonçalo moniz institute oswaldo cruz foundatio...
9853,fiocruz mg rené rachou cpqrr,grupo informática de biossistemas e genômica c...
18518,ini instituto nacional de infectologia evandr...,instituto de pesquisa clínica evandro chagas i...


In [12]:
X, y = df['Affiliations'],df['Institution']
X.shape, y.shape

((20745,), (20745,))

In [13]:
X

0                                      biblioteca  fiocruz
1        biblioteca de manguinhos avenida brasil 4365 m...
2        department of health information oswaldo cruz ...
3                        fiocruz  biblioteca de manguinhos
4        health information fundação oswaldo cruz bibli...
                               ...                        
20740    scientific computing program oswaldo cruz foun...
20741    scientific computing programoswaldo cruz found...
20742    scientific computing programoswaldo cruz found...
20743    scientific computing programme fundação oswald...
20744    serviço de jornalismo e comunicação programa d...
Name: Affiliations, Length: 20745, dtype: object

In [14]:
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/waldeyr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
palavrasIgnoradas = \
    stopwords.words('portuguese') + \
    stopwords.words('english') + \
    stopwords.words('german') + \
    stopwords.words('spanish')

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(
    max_features = 1500, # vocabulário considera top max_features ordenadas por freq do termo no texto 
    min_df = 5, # ignorar termos com frequencia menor do que x, onde 0<=x<=1
    max_df = 0.7, # ignorar termos com frequencia maior do que x, onde 0<=x<=1
    stop_words = palavrasIgnoradas
)

In [17]:
X = vectorizer.fit_transform(X).toarray()

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [19]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.1, random_state=0, shuffle=True)

# K-Nearest Neighbors

In [20]:
knn = KNeighborsClassifier()
knn.fit(X_treino, y_treino)
resultado_knn = knn.predict(X_teste)

In [21]:
print(f"Acurácia: {metrics.accuracy_score(y_teste, resultado_knn) * 100}")

Acurácia: 96.53012048192771


# DecisionTree

In [22]:
arvore = DecisionTreeClassifier(criterion='entropy' ,random_state=0)
dt = arvore.fit(X_treino, y_treino)
resultado_dt = dt.predict(X_teste)

In [23]:
print(f"Acurácia: {metrics.accuracy_score(y_teste, resultado_dt) * 100}")

Acurácia: 94.74698795180723


# RandomForest

In [24]:
floresta = RandomForestClassifier(criterion='entropy', random_state=0)
floresta.fit(X_treino, y_treino)
resultado_rd = floresta.predict(X_teste)

In [25]:
print(f"Acurácia: {metrics.accuracy_score(y_teste, resultado_rd) * 100}")

Acurácia: 97.44578313253012


# MultinomialNB

In [26]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_treino, y_treino)
resultado_nb = nb.predict(X_teste)

In [27]:
print(f"Acurácia: {accuracy_score(y_teste, resultado_nb) * 100}")

Acurácia: 95.46987951807229


# SVM Kernel Linear

In [28]:
svm_kernel_linear = svm.SVC(kernel = 'linear')
prob_linear = svm_kernel_linear.fit(X_treino, y_treino)

In [29]:
pred_svm_linear = svm_kernel_linear.predict(X_teste)
print("Acurácia =", accuracy_score(y_teste, pred_svm_linear) * 100)

Acurácia = 98.02409638554217


# SVM Kernel RBF

In [30]:
svm_kernel_rbf = svm.SVC(kernel = 'rbf')
prob_rbf = svm_kernel_rbf.fit(X_treino, y_treino)

In [31]:
pred_svm_rbf = svm_kernel_rbf.predict(X_teste)
print("Acurácia =", accuracy_score(y_teste, pred_svm_rbf) * 100)

Acurácia = 97.87951807228914


# SVM Kernel Sigmoid

In [32]:
svm_kernel_sigmoid = svm.SVC(kernel = 'sigmoid')
prob_sigmoid = svm_kernel_sigmoid.fit(X_treino, y_treino)

In [33]:
pred_svm_sigmoid = svm_kernel_sigmoid.predict(X_teste)
print("Acurácia =", accuracy_score(y_teste, pred_svm_sigmoid) * 100)

Acurácia = 97.87951807228914


# SVM Kernel Poly

In [34]:
svm_kernel_poly = svm.SVC(kernel = 'poly')
prob_poly = svm_kernel_poly.fit(X_treino, y_treino)

In [35]:
pred_svm_poly = svm_kernel_poly.predict(X_teste)
print("Acurácia =", accuracy_score(y_teste, pred_svm_poly) * 100)

Acurácia = 97.01204819277109
